In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
from api import API

from marthas_dashboard.views import (tools)

from bokeh.plotting import (figure, show)
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [153]:
from marthas_dashboard.views import create_selector_data_dict

In [154]:
selector_data = create_selector_data_dict()

In [156]:
from bokeh.models import HoverTool, ColumnDataSource, LabelSet

In [157]:
api = API()
search = {'point': '305','from': '2017-08-18', 'to': '2017-08-30'}
data = api.point_values(search['point'], search['from'], search['to'])

In [158]:
data['pointvalue'].mean()

148.44401149305557

In [159]:
average = data['pointvalue'].mean()
std = data['pointvalue'].std()

In [163]:
alerts_table = data.query(
    '(pointvalue > {}) or (pointvalue < {})'.format(
        average + std * 3,
        average - std * 3))

In [165]:
from flask import jsonify

In [171]:
alerts_table.as_matrix()

array([[nan, 5, 59, 'Hulings Hall - Electricity', 305,
        'Hulings Hall - Electricity', Timestamp('2017-08-29 15:00:00'),
        84.625, 'float', 'kWh', '2017-08-29', '15:00'],
       [nan, 5, 59, 'Hulings Hall - Electricity', 305,
        'Hulings Hall - Electricity', Timestamp('2017-08-29 16:00:00'),
        260.0, 'float', 'kWh', '2017-08-29', '16:00']], dtype=object)

In [ ]:
alerts_table.as_matrix(columns=[''])

In [179]:
alerts_table

,description,factor,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
279,NaN,5,59,Hulings Hall - Electricity,305,Hulings Hall - Electricity,2017-08-29 15:00:00,84.625,float,kWh,2017-08-29,15:00
280,NaN,5,59,Hulings Hall - Electricity,305,Hulings Hall - Electricity,2017-08-29 16:00:00,260.000,float,kWh,2017-08-29,16:00


In [180]:
cols = ['pointtimestamp', 'pointvalue']

In [192]:
smaller_alerts = alerts_table[cols]

In [196]:
smaller_alerts.to_json(orient='index')

'{"279":{"pointtimestamp":1504018800000,"pointvalue":84.625},"280":{"pointtimestamp":1504022400000,"pointvalue":260.0}}'

In [173]:
alerts_table['pointtimestamp']

279   2017-08-29 15:00:00
280   2017-08-29 16:00:00
Name: pointtimestamp, dtype: datetime64[ns]

In [113]:
average = data['pointvalue'].mean()
std = data['pointvalue'].std()
outliers = []
for index, row in data.iterrows():
    if row['pointvalue'] > average + std * 3 or row['pointvalue'] < average - std * 3:
        roundpoints = 2
        formatstring = "{0:.{1}f}"
        # we were getting values like Average: -0.0 just because we were cutting off crucial info for low vals
        # just keep increasing the precision until we display an actual non zero value
        while float(formatstring.format(average, roundpoints)) == float(0):
            roundpoints += 1
        row['avg'] = formatstring.format(average, roundpoints)
        row['std'] = formatstring.format(std, roundpoints)
        row['pointvalue'] = formatstring.format(row['pointvalue'], roundpoints)
        outliers.append(row)

print(outliers)

[description                              NaN
factor                                     5
id                                        59
name              Hulings Hall - Electricity
pointid                                  305
pointname         Hulings Hall - Electricity
pointtimestamp           2017-08-29 15:00:00
pointvalue                             84.62
returntype                             float
units                                    kWh
date                              2017-08-29
time                                   15:00
avg                                   148.44
std                                    14.46
Name: 279, dtype: object, description                              NaN
factor                                     5
id                                        59
name              Hulings Hall - Electricity
pointid                                  305
pointname         Hulings Hall - Electricity
pointtimestamp           2017-08-29 16:00:00
pointvalue                  

In [87]:
data['pointtimestamp'] = pd.to_datetime(data['pointtimestamp'])

In [88]:
source = ColumnDataSource(data)

In [104]:
hover = HoverTool(
    tooltips=[('Date', '@pointtimestamp{%F %T}'), ('Value', '$y')],
    formatters={'pointtimestamp': 'datetime'}, # use 'datetime' formatter for 'date' field
    mode='vline')

plot_tools = ['pan', 'box_zoom', 'wheel_zoom', 'save', 'reset', 'lasso_select', hover]

In [105]:
p = figure(
    plot_width=400, plot_height=400,
    x_axis_type="datetime", tools=plot_tools)
p.line(x='pointtimestamp', y='pointvalue', source=source)

GlyphRenderer(id='d6f58626-c1fe-433b-bfc6-a4ab875cb069', ...)

In [10]:
p = figure(plot_width=400, plot_height=400)
p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)

GlyphRenderer(id='be98305e-9b29-411a-92cc-2b8b50b2358e', ...)

In [7]:
show(p)

In [106]:
show(p)